In [1]:
import numpy as np
import pandas as pd
import time

from astropy import units as un
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
# The names and codes of the Vizier catalogues that have
# at least one source within the MeerKAT FoV
# of GX339-4

vizier_cats = {'GAIA EDR3': 'I/350/gaiaedr3',
               'GAIA EDR3 x TYCHO': 'I/350/tyc2tdsc',
               '2MASS': 'II/246/out',
               'WISE': 'II/311/wise',
               'AllWISE': 'II/328/allwise',
               'SkyMapper': 'II/358/smss',
               'unWISE': 'II/363/unwise',
               'DENIS': 'B/denis/denis',
               '2FGL x Radio': 'J/ApJS/217/4/table4',
               'YSO candidates': 'J/MNRAS/458/3479/clasiii',
               'Solar-type dwarfs': 'J/MNRAS/463/4210/ucac4rpm',
               'ASAS-SN variable stars': 'II/366/catalog',
               'Swift UVOT serendipitous': 'II/339/uvotssc1',
               '4XMM serendipitous': 'IX/61/xmm4r9st',
               'XMM-OM serendipitous 2019': 'II/356/xmmom41s',
               'GLADE': 'VII/281/glade2',
               '2MASX ZOA galaxy cat': 'J/MNRAS/482/5167/table3',
               'MORX': 'V/148/morx',
               'Chandra': 'IX/45/csc11',
               'Chandra CSC': 'IX/57/csc2master',
               'Swift 2SXPS': 'IX/58/2sxps',
               'Tycho': 'I/197A/tic',
               'ASAS variable stars': 'II/264/asas3',
               'VISTA': 'II/367/vhs_dr5',
               'MGPS-2': 'VIII/82/mgpscat'
               }

In [3]:
# The position uncertainties used for
# coincidence matching. 0.4 arcsec is the
# uncertainty for the MeerKAT positions
vizier_uncerts = {'GAIA EDR3 x TYCHO':0.003/60./60.,
                  'GAIA EDR3': 0.003/60./60.,
                  '2MASS': 0.1/60./60.,
                  'WISE': 0.5/60./60.,
                  'AllWISE': 0.5/60./60.,
                  'SkyMapper': 0.05/60./60.,
                  'unWISE': 0.150/60./60.,
                  'DENIS': 0.5/60./60.,
                  '2FGL x Radio': 0.002/60./60.,
                  'YSO candidates': 0.5/60./60.,
                  'Solar-type dwarfs': 0.1/60./60.,
                  'ASAS-SN variable stars': 1./60./60.,
                  'Swift UVOT serendipitous': 0.5/60./60.,
                  '4XMM serendipitous': 1.29/60./60.,
                  'XMM-OM serendipitous 2019': 0.7/60./60.,
                  'GLADE': 0.5/60./60.,
                  '2MASX ZOA galaxy cat': 1.5/60./60.,
                  'MORX': 1./60./60.,
                  'Chandra': 1.4/60./60.,
                  'Chandra CSC': 1.4/60./60.,
                  'Swift 2SXPS': 5.6/60./60.,
                  'Tycho': 0.03/60./60.,
                  'ASAS variable stars': 5./60./60.,
                  'VISTA': 0.1/60./60.,
                  'MGPS-2': 2./60./60. 
                 }

In [4]:
# Get the MeerKAT long-term variable
# source positions
gx = SkyCoord(255.70567674*un.deg,
              -48.78963475*un.deg)

source_info = pd.read_csv(('LongVariableSources.csv'))

source_coords = SkyCoord(source_info.RA_tf_deg*un.deg,
                         source_info.DEC_tf_deg*un.deg,
                         frame='icrs')

In [ ]:
# Match to the sources in the
# Gaia DR2 catalogue
radius = un.Quantity(0.4/60./60., un.deg)
for s, sc in enumerate(source_coords):
    time.sleep(5)
    gaia_search = Gaia.cone_search_async(sc, radius)
    gaia_results = gaia_search.get_results()
    
    print('********************************')
    print(s, source_info.iloc[s]['name'])
    gaia_results.pprint()
    print('********************************\n')

In [ ]:
# Get all of the sources in the FoV
# from each of the Vizier catalogues
vizier_cats_results = {}
for key, val in vizier_cats.items():
    print(key)
    time.sleep(5)
    result = Vizier.query_region(SkyCoord(ra=255.706,
                                          dec=-48.790,
                                          unit=(un.deg, un.deg),
                                          frame='icrs'),
                                 width="120m",
                                 catalog=[val])
    vizier_cats_results[key] = result

In [ ]:
# Match each MeerKAT source to each
# Vizier catalogue

for s, sc in enumerate(source_coords):
    print('Source {}'.format(s))
    print(source_info.iloc[s]['name'])
    print('RA: {0:.5f} DEC: {1:.5f}'.format(sc.ra.deg, sc.dec.deg))
    print('--------------------------------------')
    
    for key, val in vizier_cats_results.items():
        cat_info = val[0]
        
        try:
            radius = vizier_uncerts[key]
            if radius < 0.4/60./60.:
                search_radius = 0.4/60./60.
            else:
                search_radius = radius

            try:
                ra = np.array(cat_info['RA_ICRS'])
                dec = np.array(cat_info['DE_ICRS'])
                frame = 'icrs'
            except KeyError:
                try:
                    ra = np.array(cat_info['RAICRS'])
                    dec = np.array(cat_info['DEICRS'])
                    frame = 'icrs'
                except KeyError:                    
                    try:
                        ra = np.array(cat_info['_RA.icrs'])
                        dec = np.array(cat_info['_DE.icrs'])
                        frame = 'icrs'
                    except KeyError:
                        try:
                            ra = np.array(cat_info['RAJ2000'])
                            dec = np.array(cat_info['DEJ2000'])
                            frame = 'fk5'
                        except KeyError:
                            try:
                                ra = np.array(cat_info['_RA'])
                                dec = np.array(cat_info['_DE'])
                                frame = 'fk5'
                            except KeyError:
                                print(key)
            if isinstance(ra[0], float):
                cat_sc = SkyCoord(ra, dec,
                                  unit=(un.deg, un.deg),
                                  frame=frame)
            else:
                ra = ra.astype('U13')
                dec = dec.astype('U13')
                cat_sc = SkyCoord(ra, dec,
                                  unit=(un.hour, un.deg),
                                  frame=frame)
                
            
            seps = sc.separation(cat_sc)
            if np.nanmin(seps.deg) <= search_radius:
                print(key)
                print(('sep: {0:.4f} arcsec, '
                       'search radius {1:.4f} arcsec').format(np.min(seps.deg)*60.*60.,
                                                       search_radius*60.*60.))
            
        except KeyError:
            pass
    print('********************************\n')
    print('********************************')

In [ ]:
# Print the total number of sources
# in each Vizier catalogue within the
# MeerKAT FoV

print('Catalogue & Number of sources & Matching radius \\\\')
for key, val in vizier_cats_results.items():
    cat_info = val[0]

    try:
        radius = vizier_uncerts[key]
        if radius < 0.4/60./60.:
            search_radius = 0.4/60./60.
        else:
            search_radius = radius

        try:
            ra = np.array(cat_info['RA_ICRS'])
            dec = np.array(cat_info['DE_ICRS'])
            frame = 'icrs'
        except KeyError:
            try:
                ra = np.array(cat_info['RAICRS'])
                dec = np.array(cat_info['DEICRS'])
                frame = 'icrs'
            except KeyError:                    
                try:
                    ra = np.array(cat_info['_RA.icrs'])
                    dec = np.array(cat_info['_DE.icrs'])
                    frame = 'icrs'
                except KeyError:
                    try:
                        ra = np.array(cat_info['RAJ2000'])
                        dec = np.array(cat_info['DEJ2000'])
                        frame = 'fk5'
                    except KeyError:
                        try:
                            ra = np.array(cat_info['_RA'])
                            dec = np.array(cat_info['_DE'])
                            frame = 'fk5'
                        except KeyError:
                            print(key)
        if isinstance(ra[0], float):
            cat_sc = SkyCoord(ra, dec,
                              unit=(un.deg, un.deg),
                              frame=frame)
        else:
            ra = ra.astype('U13')
            dec = dec.astype('U13')
            cat_sc = SkyCoord(ra, dec,
                              unit=(un.hour, un.deg),
                              frame=frame)


        print('{0} & {1} & {2:.2f} &   \\\\'.format(key, len(cat_sc), search_radius*60.*60.))

    except KeyError:
        pass